In [127]:
import os
import json
import openai
import wandb
import pandas as pd
from tqdm import tqdm
from time import sleep
from dotenv import load_dotenv
from sklearn.metrics import accuracy_score, classification_report, f1_score

load_dotenv()

True

In [113]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [15]:
test_prompt = """i've lost my card: lostCard\ncan you activate my card?: cardActivation\ni'd like to activate my card: cardAcivation\nhelp! my card is lost: lostCard\ni can't find my card. i think i've lost it:"""

print(test_prompt)

i've lost my card: lostCard
can you activate my card?: cardActivation
i'd like to activate my card: cardAcivation
help! my card is lost: lostCard
i can't find my card. i think i've lost it:


In [16]:
response = openai.Completion.create(engine="text-ada-001",
                                    prompt=test_prompt,
                                    temperature=0,
                                    max_tokens=3)

In [17]:
response

<OpenAIObject text_completion id=cmpl-584jbypAyf8yDdbQfZ7jfD5EQQUcg at 0x7feed36abe00> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " lostCard"
    }
  ],
  "created": 1652606507,
  "id": "cmpl-584jbypAyf8yDdbQfZ7jfD5EQQUcg",
  "model": "text-ada-001",
  "object": "text_completion"
}

## Create classification -- FS10

In [18]:
from utils.data_utils import load_dataset_df

In [19]:
df = load_dataset_df()

In [27]:
sum(df[df.ds_name == 'train'].text.str.len()) / 4

129024.5

In [39]:
train_sentences = df[df.ds_name == 'train'].groupby("intent").text.apply(list).apply(lambda x: x[:10])

In [40]:
sum(train_sentences.apply(lambda x: sum([len(sent) for sent in x]))) / 4

11179.0

In [49]:
train_sentences

intent
Refund_not_showing_up                      [who should i contact if my refund doesn't go ...
activate_my_card                           [i want to activate the card., i need info on ...
age_limit                                  [can my children get an account?, would i be a...
apple_pay_or_google_pay                    [google pay isn't working. what is wrong?, can...
atm_support                                [what are the locations i can withdraw money, ...
                                                                 ...                        
virtual_card_not_working                   [i can't make purchases with my virtual card.,...
visa_or_mastercard                         [i want to get a visa and mastercard, is it po...
why_verify_identity                        [is verifying my identify important?, can i us...
wrong_amount_of_cash_received              [i wanted to withdraw $100 from the atm but i ...
wrong_exchange_rate_for_cash_withdrawal    [i was overcharged a

In [54]:
transformed_res = []

for intent, sents in train_sentences.iteritems():
    for sent in sents:
        transformed_res.append({"text": sent, "label": intent.capitalize()})

In [55]:
len(transformed_res)

770

In [58]:
print(str(transformed_res[0]))

{'text': "who should i contact if my refund doesn't go through?", 'label': 'Refund_not_showing_up'}


In [60]:
rewrite = False

if rewrite:

    with open("train_few_shot_10.jsonl", "w") as f:
        for entry in transformed_res:
            json.dump(entry, f)
            f.write('\n')

In [62]:
retrain = False

if retrain:

    res = openai.File.create(file=open("train_few_shot_10.jsonl"), purpose="classifications")
    print(res)

{
  "bytes": 80490,
  "created_at": 1652608487,
  "filename": "file",
  "id": "file-QJ6HkMO6DbnP1sD3M1gOctvU",
  "object": "file",
  "purpose": "classifications",
  "status": "uploaded",
  "status_details": null
}


In [64]:
df[df.ds_name == 'test'].head()

,text,intent,ds_name
7,my card isn't working,card_not_working,test
11,"after the transfer, the balance did not update.",balance_not_updated_after_bank_transfer,test
16,i tried to make a transfer to a beneficiary an...,beneficiary_not_allowed,test
18,i would like to change my pin.,change_pin,test
23,i need to make an immediate cancellation relat...,cancel_transfer,test


In [68]:
file_id = "file-QJ6HkMO6DbnP1sD3M1gOctvU"

result = openai.Classification.create(
    file=file_id,
    query="my card isn't working",
    search_model="ada", 
    model="ada", 
    max_examples=5
)

print(result)

{
  "completion": "cmpl-585NOovjf7Zzl1CHGrg1WUqHcLybs",
  "file": "file-QJ6HkMO6DbnP1sD3M1gOctvU",
  "label": "Card_not_working",
  "model": "ada",
  "object": "classification",
  "search_model": "ada:2020-05-03",
  "selected_examples": [
    {
      "document": 2,
      "label": "Card_not_working",
      "object": "search_result",
      "score": 270.448,
      "text": "what to do if my physical card is not working?"
    },
    {
      "document": 3,
      "label": "Virtual_card_not_working",
      "object": "search_result",
      "score": 211.841,
      "text": "this disposable virtual card is not working."
    },
    {
      "document": 1,
      "label": "Top_up_reverted",
      "object": "search_result",
      "score": 132.198,
      "text": "why isn't top-up working? my top-up money disappeared after i saw that it was successfully transferred. is it gone? where did it go?"
    },
    {
      "document": 0,
      "label": "Apple_pay_or_google_pay",
      "object": "search_result",
 

In [76]:
intents = list(df.intent.unique())

test_subset_list = []
for intent in intents:
    sub_df = df[(df.ds_name == 'test') & (df.intent == intent)].sample(10)
    test_subset_list.append(sub_df)
    
sub_test_set = pd.concat(test_subset_list)
sub_test_set.head(11)

,text,intent,ds_name
6527,how can i see where my money comes from?,verify_source_of_funds,test
6388,i need the source of my funds verified. how do...,verify_source_of_funds,test
11631,why does my money come from?,verify_source_of_funds,test
3588,what do i need to do to verify the source of m...,verify_source_of_funds,test
5295,can i verify the source of my funds?,verify_source_of_funds,test
2592,i would like to verify the source of my money,verify_source_of_funds,test
7494,what is the source of my funds.,verify_source_of_funds,test
5808,how can i check the source of funds?,verify_source_of_funds,test
5589,how do i become aware of where my funds come f...,verify_source_of_funds,test
10425,how can i check the source for my funds?,verify_source_of_funds,test


In [119]:
sub_test_set.to_json("dataset/sub_test_set.json")

In [87]:
def classify_fs10(text):
    file_id = "file-QJ6HkMO6DbnP1sD3M1gOctvU"

    result = openai.Classification.create(
        file=file_id,
        query=text,
        search_model="ada", 
        model="ada", 
        max_examples=5
    )
        
    return result

In [97]:
prediction_responses = []

for i, rec in tqdm(sub_test_set.iterrows()):
    text = rec.text
    orig_intent = rec.intent
    
    pred_response = classify_fs10(text)
    sleep(2)
    
    prediction_responses.append({
        "query_text": text,
        "true_intent": orig_intent,
        "pred_resp": pred_response
    })

770it [58:15,  4.54s/it]


In [102]:
res_df = pd.DataFrame(prediction_responses)
res_df['true_intent'] = res_df['true_intent'].str.capitalize()
res_df['pred_intent'] = res_df.pred_resp.apply(lambda x: x['label'])
res_df.head()

,query_text,true_intent,pred_resp,pred_intent
0,how can i see where my money comes from?,Verify_source_of_funds,{'completion': 'cmpl-585zu4z3uhktf6W39HDhp9DCi...,Wrong_amount_of_cash_received
1,i need the source of my funds verified. how do...,Verify_source_of_funds,{'completion': 'cmpl-585zymSbu6rTMBMD5eHUyUR7r...,Verify_source_of_funds
2,why does my money come from?,Verify_source_of_funds,{'completion': 'cmpl-5860383EdxIrOi4RuZMPPse12...,Declined_cash_withdrawal
3,what do i need to do to verify the source of m...,Verify_source_of_funds,{'completion': 'cmpl-58608bhMmqfVdJNSbdRrTkNjd...,Verify_source_of_funds
4,can i verify the source of my funds?,Verify_source_of_funds,{'completion': 'cmpl-5860C7i1rK5ndjStBaakdVgm6...,Verify_source_of_funds


In [106]:
accuracy_score(res_df['true_intent'].tolist(), res_df['pred_intent'].tolist())

0.4857142857142857

In [107]:
print(classification_report(res_df['true_intent'].tolist(), res_df['pred_intent'].tolist()))

                                                  precision    recall  f1-score   support

                                Activate_my_card       1.00      0.40      0.57        10
                                       Age_limit       1.00      0.80      0.89        10
                         Apple_pay_or_google_pay       0.64      0.90      0.75        10
                                     Atm_support       0.86      0.60      0.71        10
                                Automatic_top_up       0.86      0.60      0.71        10
         Balance_not_updated_after_bank_transfer       0.42      0.50      0.45        10
Balance_not_updated_after_cheque_or_cash_deposit       0.50      0.10      0.17        10
                         Beneficiary_not_allowed       1.00      0.20      0.33        10
                                 Cancel_transfer       0.75      0.60      0.67        10
                            Card_about_to_expire       0.75      0.60      0.67        10
         

/Users/somanagy/Library/Caches/pypoetry/virtualenvs/intent-detection-sBxw7ymE-py3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/somanagy/Library/Caches/pypoetry/virtualenvs/intent-detection-sBxw7ymE-py3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/somanagy/Library/Caches/pypoetry/virtualenvs/intent-detection-sBxw7ymE-py3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with 

In [116]:
res_df.to_json("GPT3_few_shot_prediction.json")

## Create full classification

In [114]:
rewrite = True
retrain = True

train_sentences = df[df.ds_name == 'train'].groupby("intent").text.apply(list)

if rewrite:

    with open("train_full.jsonl", "w") as f:
        for entry in transformed_res:
            json.dump(entry, f)
            f.write('\n')

if retrain:

    res = openai.File.create(file=open("train_full.jsonl"), purpose="classifications")
    print(res)

{
  "bytes": 80490,
  "created_at": 1652616313,
  "filename": "file",
  "id": "file-2KWeai6RUbruXbl2xPF9J3uG",
  "object": "file",
  "purpose": "classifications",
  "status": "uploaded",
  "status_details": null
}


In [117]:
def classify_full(text):
    file_id = "file-2KWeai6RUbruXbl2xPF9J3uG"

    result = openai.Classification.create(
        file=file_id,
        query=text,
        search_model="ada", 
        model="curie", 
        max_examples=25
    )
        
    return result

In [120]:
prediction_responses = []

for i, rec in tqdm(sub_test_set.iterrows()):
    text = rec.text
    orig_intent = rec.intent
    
    pred_response = classify_full(text)
    sleep(1.5)
    
    prediction_responses.append({
        "query_text": text,
        "true_intent": orig_intent,
        "pred_resp": pred_response
    })

770it [36:39,  2.86s/it]


In [121]:
res_df = pd.DataFrame(prediction_responses)
res_df['true_intent'] = res_df['true_intent'].str.capitalize()
res_df['pred_intent'] = res_df.pred_resp.apply(lambda x: x['label'])
res_df.head()

,query_text,true_intent,pred_resp,pred_intent
0,how can i see where my money comes from?,Verify_source_of_funds,{'completion': 'cmpl-587X16VEYtTQxSfOpPVjhdvPO...,Verify_source_of_funds
1,i need the source of my funds verified. how do...,Verify_source_of_funds,{'completion': 'cmpl-587X5qbOP2GFkIW3eYZ0T129B...,Verify_source_of_funds
2,why does my money come from?,Verify_source_of_funds,{'completion': 'cmpl-587X9FLPl77JG4nwrW3VtBkph...,Verify_source_of_funds
3,what do i need to do to verify the source of m...,Verify_source_of_funds,{'completion': 'cmpl-587XC7IclWo1b5oCNYt0ElbEN...,Verify_source_of_funds
4,can i verify the source of my funds?,Verify_source_of_funds,{'completion': 'cmpl-587XFFq5hJXWOjYD1F8e07m7Q...,Verify_source_of_funds


In [122]:
accuracy_score(res_df['true_intent'].tolist(), res_df['pred_intent'].tolist())

0.7714285714285715

In [123]:
print(classification_report(res_df['true_intent'].tolist(), res_df['pred_intent'].tolist()))

                                                  precision    recall  f1-score   support

                                Activate_my_card       1.00      0.90      0.95        10
                                       Age_limit       1.00      0.90      0.95        10
                         Apple_pay_or_google_pay       0.91      1.00      0.95        10
                                     Atm_support       1.00      0.90      0.95        10
                                Automatic_top_up       1.00      0.90      0.95        10
         Balance_not_updated_after_bank_transfer       0.83      0.50      0.62        10
Balance_not_updated_after_cheque_or_cash_deposit       0.88      0.70      0.78        10
                         Beneficiary_not_allowed       1.00      0.60      0.75        10
                                 Cancel_transfer       0.75      0.90      0.82        10
                            Card_about_to_expire       0.80      0.80      0.80        10
         

/Users/somanagy/Library/Caches/pypoetry/virtualenvs/intent-detection-sBxw7ymE-py3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/somanagy/Library/Caches/pypoetry/virtualenvs/intent-detection-sBxw7ymE-py3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/somanagy/Library/Caches/pypoetry/virtualenvs/intent-detection-sBxw7ymE-py3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with 

In [124]:
res_df.to_json("GPT3_full_prediction.json")

In [128]:
wandb.init(project="intent-detection", entity='nsoma')

wandb.run.name = "GPT-3 sem-search classification"

wandb.log({"test_accuracy": accuracy_score(res_df['true_intent'].tolist(), res_df['pred_intent'].tolist()),
           "test_f1_score": f1_score(res_df['true_intent'].tolist(), res_df['pred_intent'].tolist(), average='weighted')})

wandb.finish()

wandb: Currently logged in as: nsoma. Use `wandb login --relogin` to force relogin


test_accuracy,▁
test_f1_score,▁
test_accuracy,0.77143
test_f1_score,0.78797


## Fine-tune GPT-3

In [129]:
transformed_res = []

for intent, sents in train_sentences.iteritems():
    for sent in sents:
        transformed_res.append({"prompt": sent, "completion": intent})


with open("fine_tune_data.jsonl", "w") as f:
        for entry in transformed_res:
            json.dump(entry, f)
            f.write('\n')

In [135]:
!head fine_tune_data.jsonl

{"prompt": "who should i contact if my refund doesn't go through?", "completion": "Refund_not_showing_up"}
{"prompt": "please help me.  i am still waiting on a refund.", "completion": "Refund_not_showing_up"}
{"prompt": "when should i receive my refund", "completion": "Refund_not_showing_up"}
{"prompt": "i am missing my refund.", "completion": "Refund_not_showing_up"}
{"prompt": "when will my returned transaction show up on my account?", "completion": "Refund_not_showing_up"}
{"prompt": "i have not received a refund.", "completion": "Refund_not_showing_up"}
{"prompt": "where is my refund? it appears to be missing.", "completion": "Refund_not_showing_up"}
{"prompt": "i was suppose to be getting a refund for something i purchased a while ago, but it still hasnt been applied to my account. when will i get my money back?", "completion": "Refund_not_showing_up"}
{"prompt": "i can't see my refund in my history.", "completion": "Refund_not_showing_up"}
{"prompt": "i requested a refund but can

In [137]:
!openai tools fine_tunes.prepare_data -f fine_tune_data.jsonl -q

Analyzing...

- Your file contains 8622 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- There are 4 duplicated prompt-completion sets. These are rows: [545, 1827, 2753, 5721]
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://beta.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts empty
- The completion should start with a whitespace character (` `). This tends to produce better results due to the tokenization we us

In [143]:
!openai api fine_tunes.create

usage: openai api fine_tunes.create [-h] -t TRAINING_FILE [-v VALIDATION_FILE]
                                    [--no_check_if_files_exist] [-m MODEL]
                                    [--suffix SUFFIX] [--no_follow]
                                    [--n_epochs N_EPOCHS]
                                    [--batch_size BATCH_SIZE]
                                    [--learning_rate_multiplier LEARNING_RATE_MULTIPLIER]
                                    [--prompt_loss_weight PROMPT_LOSS_WEIGHT]
                                    [--compute_classification_metrics]
                                    [--classification_n_classes CLASSIFICATION_N_CLASSES]
                                    [--classification_positive_class CLASSIFICATION_POSITIVE_CLASS]
                                    [--classification_betas CLASSIFICATION_BETAS [CLASSIFICATION_BETAS ...]]
openai api fine_tunes.create: error: the following arguments are required: -t/--training_file


In [148]:
!openai api fine_tunes.create -t "fine_tune_data_prepared_train.jsonl" -v "fine_tune_data_prepared_valid.jsonl" --no_check_if_files_exist -m ada

Upload progress: 100%|███████████████████████| 877k/877k [00:00<00:00, 321Mit/s]
Uploaded file from fine_tune_data_prepared_train.jsonl: file-zRfjk0qOcgaHeVBNj7m3vzG1
Upload progress: 100%|███████████████████████| 115k/115k [00:00<00:00, 101Mit/s]
Uploaded file from fine_tune_data_prepared_valid.jsonl: file-uQeC3zYZ3DVSnOGGuRraK3TG
Created fine-tune: ft-Dc0iIRPBKBEWcqiBdnZE399e
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2022-05-15 15:43:26] Created fine-tune: ft-Dc0iIRPBKBEWcqiBdnZE399e
[2022-05-15 15:46:38] Fine-tune costs $0.26
[2022-05-15 15:46:39] Fine-tune enqueued. Queue number: 0
[2022-05-15 15:46:43] Fine-tune started
[2022-05-15 15:52:49] Completed epoch 1/4

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-Dc0iIRPBKBEWcqiBdnZE399e



In [149]:
!openai api fine_tunes.follow -i ft-Dc0iIRPBKBEWcqiBdnZE399e

[2022-05-15 15:43:26] Created fine-tune: ft-Dc0iIRPBKBEWcqiBdnZE399e
[2022-05-15 15:46:38] Fine-tune costs $0.26
[2022-05-15 15:46:39] Fine-tune enqueued. Queue number: 0
[2022-05-15 15:46:43] Fine-tune started
[2022-05-15 15:52:49] Completed epoch 1/4
[2022-05-15 15:58:38] Completed epoch 2/4
[2022-05-15 16:04:28] Completed epoch 3/4
[2022-05-15 16:10:18] Completed epoch 4/4
[2022-05-15 16:10:41] Uploaded model: ada:ft-personal-2022-05-15-14-10-39
[2022-05-15 16:10:44] Uploaded result file: file-NiTfLhrTFMyAkpRTi8tjo7RB
[2022-05-15 16:10:45] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m ada:ft-personal-2022-05-15-14-10-39 -p <YOUR_PROMPT>


In [151]:
!openai wandb sync

wandb: Currently logged in as: nsoma. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.12.16
wandb: Run data is saved locally in /Users/somanagy/Desktop/repos/intent-detection/wandb/run-20220515_161416-ft-Dc0iIRPBKBEWcqiBdnZE399e
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ft-Dc0iIRPBKBEWcqiBdnZE399e
wandb: ⭐️ View project at https://wandb.ai/nsoma/GPT-3
wandb: 🚀 View run at https://wandb.ai/nsoma/GPT-3/runs/ft-Dc0iIRPBKBEWcqiBdnZE399e
File file-zRfjk0qOcgaHeVBNj7m3vzG1 could not be retrieved. Make sure you are allowed to download training/validation files
File file-uQeC3zYZ3DVSnOGGuRraK3TG could not be retrieved. Make sure you are allowed to download training/validation files
wandb: Waiting for W&B process to finish... (success).
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:             elapsed_examples ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb: 

In [159]:
!openai api fine_tunes.results -i ft-Dc0iIRPBKBEWcqiBdnZE399e > result.csv

In [161]:
results = pd.read_csv('result.csv')
results

,step,elapsed_tokens,elapsed_examples,training_loss,training_sequence_accuracy,training_token_accuracy,validation_loss,validation_sequence_accuracy,validation_token_accuracy
0,1,392,8,0.691199,0.000,0.274510,0.389372,0.0,0.574074
1,2,848,16,0.470169,0.000,0.468085,NaN,NaN,NaN
2,3,1176,24,0.630532,0.000,0.609375,NaN,NaN,NaN
3,4,1504,32,0.514540,0.000,0.637931,NaN,NaN,NaN
4,5,1896,40,0.373151,0.000,0.589744,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
3805,3806,1286320,30448,0.050984,1.000,1.000000,NaN,NaN,NaN
3806,3807,1286584,30456,0.062177,0.875,0.976744,NaN,NaN,NaN
3807,3808,1286912,30464,0.056756,1.000,1.000000,NaN,NaN,NaN
3808,3809,1287368,30472,0.061187,1.000,1.000000,NaN,NaN,NaN


In [150]:
!openai api completions.create -m ada:ft-personal-2022-05-15-14-10-39 -p "can i verify the source of my funds? ->"

can i verify the source of my funds? -> verify_source_of_funds_verification_ challans_had

In [152]:
ft_model = "ada:ft-personal-2022-05-15-14-10-39"

In [ ]:
test_prompt = "can i verify the source of my funds?"

In [157]:
res = openai.Completion.create(model=ft_model, prompt=test_prompt + ' ->', max_tokens=15, temperature=0)
res

<OpenAIObject text_completion id=cmpl-589MmegpkBmHYMLXUbAM9Srhh9Jtw at 0x7feec30fed10> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " lost_or_stolen_card_not_working_anymore"
    }
  ],
  "created": 1652624312,
  "id": "cmpl-589MmegpkBmHYMLXUbAM9Srhh9Jtw",
  "model": "ada:ft-personal-2022-05-15-14-10-39",
  "object": "text_completion"
}

### Fine-tune a Curie model

In [162]:
!openai api fine_tunes.create -t "fine_tune_data_prepared_train.jsonl" -v "fine_tune_data_prepared_valid.jsonl" --no_check_if_files_exist -m curie

Upload progress: 100%|███████████████████████| 877k/877k [00:00<00:00, 367Mit/s]
Uploaded file from fine_tune_data_prepared_train.jsonl: file-vdE2hDOtN4nYHLc7fujwUmtn
Upload progress: 100%|███████████████████████| 115k/115k [00:00<00:00, 143Mit/s]
Uploaded file from fine_tune_data_prepared_valid.jsonl: file-b1B5QMsbYZVXRDXN6o04OF4w
Created fine-tune: ft-QoR0ZWwaUq6liNIOyPAD1PNR
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2022-05-16 20:10:04] Created fine-tune: ft-QoR0ZWwaUq6liNIOyPAD1PNR

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-QoR0ZWwaUq6liNIOyPAD1PNR



In [164]:
!openai api fine_tunes.follow -i ft-QoR0ZWwaUq6liNIOyPAD1PNR

[2022-05-16 20:10:04] Created fine-tune: ft-QoR0ZWwaUq6liNIOyPAD1PNR
[2022-05-16 20:26:47] Fine-tune costs $1.96
[2022-05-16 20:26:48] Fine-tune enqueued. Queue number: 0
[2022-05-16 20:38:01] Fine-tune started
[2022-05-16 20:59:56] Completed epoch 2/4
[2022-05-16 21:08:39] Completed epoch 3/4
[2022-05-16 21:17:21] Completed epoch 4/4
[2022-05-16 21:17:48] Uploaded model: curie:ft-personal-2022-05-16-19-17-46
[2022-05-16 21:17:51] Uploaded result file: file-XlfhpQqaXrtQUTesENms3LpH
[2022-05-16 21:17:52] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m curie:ft-personal-2022-05-16-19-17-46 -p <YOUR_PROMPT>


In [166]:
!openai api completions.create -m curie:ft-personal-2022-05-16-19-17-46 -p "can i verify the source of my funds? ->"

can i verify the source of my funds? -> verify_source_of_funds_support_not_being_allowed

### Use numerical class labels

In [168]:
transformed_res = []

for ind, (intent, sents) in enumerate(train_sentences.iteritems()):
    for sent in sents:
        transformed_res.append({"prompt": sent, "completion": f" {ind}"})


with open("fine_tune_data_num.jsonl", "w") as f:
        for entry in transformed_res:
            json.dump(entry, f)
            f.write('\n')

In [169]:
!head fine_tune_data_num.jsonl

{"prompt": "who should i contact if my refund doesn't go through?", "completion": " 0"}
{"prompt": "please help me.  i am still waiting on a refund.", "completion": " 0"}
{"prompt": "when should i receive my refund", "completion": " 0"}
{"prompt": "i am missing my refund.", "completion": " 0"}
{"prompt": "when will my returned transaction show up on my account?", "completion": " 0"}
{"prompt": "i have not received a refund.", "completion": " 0"}
{"prompt": "where is my refund? it appears to be missing.", "completion": " 0"}
{"prompt": "i was suppose to be getting a refund for something i purchased a while ago, but it still hasnt been applied to my account. when will i get my money back?", "completion": " 0"}
{"prompt": "i can't see my refund in my history.", "completion": " 0"}
{"prompt": "i requested a refund but can't see it.", "completion": " 0"}


In [170]:
!openai tools fine_tunes.prepare_data -f fine_tune_data_num.jsonl -q

Analyzing...

- Your file contains 8622 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- There are 4 duplicated prompt-completion sets. These are rows: [545, 1827, 2753, 5721]
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://beta.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts empty

Based on the analysis we will perform the following actions:
- [Recommended] Remove 4 duplicate rows [Y/n]: Y
- [Recommended] Add 

In [171]:
!openai api fine_tunes.create -t "fine_tune_data_num_prepared_train.jsonl" -v "fine_tune_data_num_prepared_valid.jsonl" --compute_classification_metrics --classification_n_classes 77  -m curie

Upload progress: 100%|███████████████████████| 729k/729k [00:00<00:00, 322Mit/s]
Uploaded file from fine_tune_data_num_prepared_train.jsonl: file-OohoSsfuEfDi5NpDjL9QFWk4
Upload progress: 100%|█████████████████████| 96.2k/96.2k [00:00<00:00, 145Mit/s]
Uploaded file from fine_tune_data_num_prepared_valid.jsonl: file-6XIHAbKJ8rTaM7h3b1FogUMC
Created fine-tune: ft-pYuHw6vc6DwCm0ZDWc5yAzcC
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2022-05-16 21:47:06] Created fine-tune: ft-pYuHw6vc6DwCm0ZDWc5yAzcC

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-pYuHw6vc6DwCm0ZDWc5yAzcC



In [181]:
mapping_df = pd.DataFrame(train_sentences)
mapping_df["ind"] = list(range(77))
mapping_df.reset_index()[['intent', 'ind']].to_json("label_num_mapping.json")

In [183]:
!openai api fine_tunes.follow -i ft-pYuHw6vc6DwCm0ZDWc5yAzcC

[2022-05-16 21:47:06] Created fine-tune: ft-pYuHw6vc6DwCm0ZDWc5yAzcC
[2022-05-16 22:02:31] Fine-tune costs $1.46
[2022-05-16 22:02:31] Fine-tune enqueued. Queue number: 0
[2022-05-16 22:02:34] Fine-tune started
[2022-05-16 22:12:18] Completed epoch 1/4
[2022-05-16 22:21:33] Completed epoch 2/4
[2022-05-16 22:30:46] Completed epoch 3/4
[2022-05-16 22:39:55] Completed epoch 4/4
[2022-05-16 22:40:36] Uploaded model: curie:ft-personal-2022-05-16-20-40-34
[2022-05-16 22:40:40] Uploaded result file: file-s7bUz51bJrKHahSgX89TjfTj
[2022-05-16 22:40:40] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m curie:ft-personal-2022-05-16-20-40-34 -p <YOUR_PROMPT>


In [184]:
!openai api completions.create -m curie:ft-personal-2022-05-16-20-40-34 -p "can i verify the source of my funds? ->"

can i verify the source of my funds? -> 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70

In [186]:
mapping_df[mapping_df.ind == 70]

,text,ind
intent,,
verify_source_of_funds,[i would like to know how to work out where my...,70


In [188]:
ft_model = "curie:ft-personal-2022-05-16-20-40-34"

res = openai.Completion.create(model=ft_model, prompt="can i verify the source of my funds?" + ' ->', max_tokens=1, temperature=0)
res

<OpenAIObject text_completion id=cmpl-58cLCnU65xMEaHy95UFhX1fBax0dx at 0x7feec41b0810> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " 70"
    }
  ],
  "created": 1652735690,
  "id": "cmpl-58cLCnU65xMEaHy95UFhX1fBax0dx",
  "model": "curie:ft-personal-2022-05-16-20-40-34",
  "object": "text_completion"
}

In [189]:
!openai wandb sync

wandb: Currently logged in as: nsoma. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.12.16
wandb: Run data is saved locally in /Users/somanagy/Desktop/repos/intent-detection/wandb/run-20220516_231522-ft-QoR0ZWwaUq6liNIOyPAD1PNR
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ft-QoR0ZWwaUq6liNIOyPAD1PNR
wandb: ⭐️ View project at https://wandb.ai/nsoma/GPT-3
wandb: 🚀 View run at https://wandb.ai/nsoma/GPT-3/runs/ft-QoR0ZWwaUq6liNIOyPAD1PNR
File file-vdE2hDOtN4nYHLc7fujwUmtn could not be retrieved. Make sure you are allowed to download training/validation files
File file-b1B5QMsbYZVXRDXN6o04OF4w could not be retrieved. Make sure you are allowed to download training/validation files
wandb: Waiting for W&B process to finish... (success).
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:             elapsed_examples ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb: 

In [191]:
def classify_finetuned(text):

    result = openai.Completion.create(
        model="curie:ft-personal-2022-05-16-20-40-34",
        prompt=text + " ->",
        max_tokens=1
    )
        
    return result

In [192]:
classify_finetuned("can i verify the source of my funds?")

<OpenAIObject text_completion id=cmpl-58cP7ZWwCsp9ERpzNYI0fHHLUiPbC at 0x7feebf753810> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " 70"
    }
  ],
  "created": 1652735933,
  "id": "cmpl-58cP7ZWwCsp9ERpzNYI0fHHLUiPbC",
  "model": "curie:ft-personal-2022-05-16-20-40-34",
  "object": "text_completion"
}

In [193]:
prediction_responses = []

for i, rec in tqdm(sub_test_set.iterrows()):
    text = rec.text
    orig_intent = rec.intent
    
    pred_response = classify_finetuned(text)
    sleep(2)
    
    prediction_responses.append({
        "query_text": text,
        "true_intent": orig_intent,
        "pred_resp": pred_response
    })

770it [47:20,  3.69s/it]


In [197]:
int(" 0".lstrip())

0

In [214]:
def pred_to_label(pred):
    try:
        ind = int(pred.lstrip())
        return mapping_df[mapping_df.ind == ind].index[0]
    except:
        return "NA"
    
pred_to_label(" 70")

'verify_source_of_funds'

In [201]:
prediction_responses[0]

{'query_text': 'how can i see where my money comes from?',
 'true_intent': 'verify_source_of_funds',
 'pred_resp': <OpenAIObject text_completion id=cmpl-58cQBY45WXgyAVx66Ss0uYWSEm5Sg at 0x7feebf74aef0> JSON: {
   "choices": [
     {
       "finish_reason": "length",
       "index": 0,
       "logprobs": null,
       "text": " 70"
     }
   ],
   "created": 1652735999,
   "id": "cmpl-58cQBY45WXgyAVx66Ss0uYWSEm5Sg",
   "model": "curie:ft-personal-2022-05-16-20-40-34",
   "object": "text_completion"
 }}

In [215]:
res_df = pd.DataFrame(prediction_responses)
res_df['true_intent'] = res_df['true_intent']
res_df['pred_intent'] = res_df.pred_resp.apply(lambda x: pred_to_label(x['choices'][0]['text']))
res_df.head()

,query_text,true_intent,pred_resp,pred_intent
0,how can i see where my money comes from?,verify_source_of_funds,"{'id': 'cmpl-58cQBY45WXgyAVx66Ss0uYWSEm5Sg', '...",verify_source_of_funds
1,i need the source of my funds verified. how do...,verify_source_of_funds,"{'id': 'cmpl-58cQIFdiZsOToqpklRekYJAdteXjQ', '...",verify_source_of_funds
2,why does my money come from?,verify_source_of_funds,"{'id': 'cmpl-58cQLvysGXZOUdy0BfXaHUaVrUpWV', '...",verify_source_of_funds
3,what do i need to do to verify the source of m...,verify_source_of_funds,"{'id': 'cmpl-58cQOveVJ1slDjQmgvKd6x0R0rnl3', '...",verify_source_of_funds
4,can i verify the source of my funds?,verify_source_of_funds,"{'id': 'cmpl-58cQZYDS33YOExdZd7fnWWdzXEEBR', '...",verify_source_of_funds


In [216]:
accuracy_score(res_df['true_intent'].tolist(), res_df['pred_intent'].tolist())

0.9181818181818182

In [217]:
print(classification_report(res_df['true_intent'].tolist(), res_df['pred_intent'].tolist()))

                                                  precision    recall  f1-score   support

                           Refund_not_showing_up       1.00      0.80      0.89        10
                                activate_my_card       1.00      1.00      1.00        10
                                       age_limit       1.00      1.00      1.00        10
                         apple_pay_or_google_pay       1.00      0.90      0.95        10
                                     atm_support       1.00      1.00      1.00        10
                                automatic_top_up       0.82      0.90      0.86        10
         balance_not_updated_after_bank_transfer       0.62      0.80      0.70        10
balance_not_updated_after_cheque_or_cash_deposit       0.83      1.00      0.91        10
                         beneficiary_not_allowed       0.89      0.80      0.84        10
                                 cancel_transfer       1.00      0.90      0.95        10
         

In [218]:
res_df.to_json("GPT3_full_finetuned.json")

In [219]:
wandb.init(project="intent-detection", entity='nsoma')

wandb.run.name = "GPT-3 fine-tuned classification"

wandb.log({"test_accuracy": accuracy_score(res_df['true_intent'].tolist(), res_df['pred_intent'].tolist()),
           "test_f1_score": f1_score(res_df['true_intent'].tolist(), res_df['pred_intent'].tolist(), average='weighted')})

wandb.finish()

test_accuracy,▁
test_f1_score,▁
test_accuracy,0.91818
test_f1_score,0.91821


## Full training dataset

In [229]:
from random import shuffle

In [230]:
valid_sentences = df[df.ds_name == 'valid'].groupby("intent").text.apply(list)

transformed_res = []

for intent, sents in valid_sentences.iteritems():
    ind = mapping_df.loc[intent].ind
    for sent in sents:
        transformed_res.append({"prompt": sent, "completion": f" {ind}"})

shuffle(transformed_res)

with open("full_original_valid_gpt.jsonl", "w") as f:
        for entry in transformed_res:
            json.dump(entry, f)
            f.write('\n')

In [232]:
!openai api fine_tunes.create -t "full_original_train_gpt.jsonl" -v "full_original_valid_gpt.jsonl" --compute_classification_metrics --classification_n_classes 77  -m curie

Upload progress: 100%|███████████████████████| 825k/825k [00:00<00:00, 255Mit/s]
Uploaded file from full_original_train_gpt.jsonl: file-BICtTHuvukXUORWkFO87NI93
Upload progress: 100%|███████████████████████| 142k/142k [00:00<00:00, 204Mit/s]
Uploaded file from full_original_valid_gpt.jsonl: file-7aX3oOqATaR0RF4F5flOuT8s
Created fine-tune: ft-tz7iA9qmnCHpX2lmqBNYqhkC
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2022-05-17 09:27:49] Created fine-tune: ft-tz7iA9qmnCHpX2lmqBNYqhkC
[2022-05-17 09:35:22] Fine-tune costs $1.65
[2022-05-17 09:35:22] Fine-tune enqueued. Queue number: 0
[2022-05-17 09:35:27] Fine-tune started
[2022-05-17 09:41:12] Completed epoch 1/4

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-tz7iA9qmnCHpX2lmqBNYqhkC



In [233]:
!openai api fine_tunes.follow -i ft-tz7iA9qmnCHpX2lmqBNYqhkC

[2022-05-17 09:27:49] Created fine-tune: ft-tz7iA9qmnCHpX2lmqBNYqhkC
[2022-05-17 09:35:22] Fine-tune costs $1.65
[2022-05-17 09:35:22] Fine-tune enqueued. Queue number: 0
[2022-05-17 09:35:27] Fine-tune started
[2022-05-17 09:41:12] Completed epoch 1/4
[2022-05-17 09:51:53] Completed epoch 3/4
[2022-05-17 10:01:38] Uploaded model: curie:ft-personal-2022-05-17-08-01-36
[2022-05-17 10:01:41] Uploaded result file: file-rRbDlgcS6wuGK597cSaMv9CR
[2022-05-17 10:01:41] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m curie:ft-personal-2022-05-17-08-01-36 -p <YOUR_PROMPT>


In [237]:
!openai api completions.create -m curie:ft-personal-2022-05-17-08-01-36 -p "can i verify the source of my funds? ->"

can i verify the source of my funds? -> 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70

In [238]:
full_test_set = df[(df.ds_name == 'test')]
len(full_test_set)

3080

In [239]:
!openai wandb sync

wandb: Currently logged in as: nsoma. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.12.16
wandb: Run data is saved locally in /Users/somanagy/Desktop/repos/intent-detection/wandb/run-20220517_132647-ft-tz7iA9qmnCHpX2lmqBNYqhkC
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ft-tz7iA9qmnCHpX2lmqBNYqhkC
wandb: ⭐️ View project at https://wandb.ai/nsoma/GPT-3
wandb: 🚀 View run at https://wandb.ai/nsoma/GPT-3/runs/ft-tz7iA9qmnCHpX2lmqBNYqhkC
File file-BICtTHuvukXUORWkFO87NI93 could not be retrieved. Make sure you are allowed to download training/validation files
File file-7aX3oOqATaR0RF4F5flOuT8s could not be retrieved. Make sure you are allowed to download training/validation files
wandb: Waiting for W&B process to finish... (success).
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:          classification/accuracy ▄█▁▃
wandb: classification/weighted_f1_score

In [240]:
full_test_set.tail(5)

,text,intent,ds_name
13204,i would like to get help from someone with act...,activate_my_card,test
13211,i got 2 transfer fees and i thought transfers ...,transfer_fee_charged,test
13225,i was wondering if there were any discounts of...,exchange_charge,test
13232,why is there a random withdrawal in my app?,cash_withdrawal_not_recognised,test
13239,i don't live in the uk. can i still get a card?,country_support,test


In [241]:
def classify_finetuned(text):

    result = openai.Completion.create(
        model="curie:ft-personal-2022-05-17-08-01-36",
        prompt=text + " ->",
        max_tokens=1
    )
        
    return result


classify_finetuned("i don't live in the uk. can i still get a card?")

<OpenAIObject text_completion id=cmpl-58pi1iKbCPDuKh6gsMROb5eprxEyu at 0x7feed898d810> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " 25"
    }
  ],
  "created": 1652787077,
  "id": "cmpl-58pi1iKbCPDuKh6gsMROb5eprxEyu",
  "model": "curie:ft-personal-2022-05-17-08-01-36",
  "object": "text_completion"
}

In [242]:
pred_to_label(" 25")

'country_support'

In [ ]:
# prediction_responses = []

In [268]:
from openai.error import RateLimitError

In [269]:
prediction_responses = prediction_responses

for i, rec in tqdm(full_test_set.iloc[len(prediction_responses):].iterrows()):
    text = rec.text
    orig_intent = rec.intent
    
    pred_response = classify_finetuned(text)
    sleep(1.5)
    
    prediction_responses.append({
        "query_text": text,
        "true_intent": orig_intent,
        "pred_resp": pred_response
    })

0it [00:16, ?it/s]


RateLimitError: The server is currently overloaded with other requests. Sorry about that! You can retry your request, or contact support@openai.com if the error persists.

In [261]:
len(prediction_responses)

1077

In [270]:
res_df = pd.DataFrame(prediction_responses)
res_df['true_intent'] = res_df['true_intent']
res_df['pred_intent'] = res_df.pred_resp.apply(lambda x: pred_to_label(x['choices'][0]['text']))
res_df.head()

,query_text,true_intent,pred_resp,pred_intent
0,my card isn't working,card_not_working,"{'id': 'cmpl-58pjY60nWtea0bbZIgEv3Duq0Jlxm', '...",card_not_working
1,"after the transfer, the balance did not update.",balance_not_updated_after_bank_transfer,"{'id': 'cmpl-58pjbjAymJFbybiEh3ALKJWMZF7bO', '...",balance_not_updated_after_bank_transfer
2,i tried to make a transfer to a beneficiary an...,beneficiary_not_allowed,"{'id': 'cmpl-58pjdSChDeaLOxf3UY8y1K80VLeEh', '...",beneficiary_not_allowed
3,i would like to change my pin.,change_pin,"{'id': 'cmpl-58pjfw5Rygevc8abw3o0sC0MsaldH', '...",change_pin
4,i need to make an immediate cancellation relat...,cancel_transfer,"{'id': 'cmpl-58pjlMWgQOBV9pqPvKPbSYqSIULbO', '...",cancel_transfer


In [272]:
accuracy_score(res_df['true_intent'].tolist(), res_df['pred_intent'].tolist())

0.9052924791086351

In [263]:
from collections import Counter

In [267]:
Counter([x['true_intent'] for x in prediction_responses])

Counter({'card_not_working': 18,
         'balance_not_updated_after_bank_transfer': 12,
         'beneficiary_not_allowed': 11,
         'change_pin': 9,
         'cancel_transfer': 13,
         'transfer_timing': 10,
         'top_up_limits': 18,
         'exchange_charge': 16,
         'terminate_account': 18,
         'pending_card_payment': 13,
         'virtual_card_not_working': 15,
         'transfer_not_received_by_recipient': 13,
         'receiving_money': 13,
         'balance_not_updated_after_cheque_or_cash_deposit': 11,
         'top_up_by_bank_transfer_charge': 14,
         'activate_my_card': 16,
         'card_payment_wrong_exchange_rate': 16,
         'topping_up_by_card': 12,
         'cash_withdrawal_charge': 19,
         'card_linking': 17,
         'contactless_not_working': 12,
         'unable_to_verify_identity': 12,
         'fiat_currency_support': 14,
         'pending_cash_withdrawal': 8,
         'card_arrival': 11,
         'card_payment_not_recognised':